In [18]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#use tf gpu 1.9.0, 1.12.0 gives cudnn version mismatch error

In [2]:
#check data
def load_text_captions(filename):
    file = open(filename, 'r', encoding="ISO-8859-1")
    text = file.read()
    file.close()
    return text

filename = "dataset/30k_captions.txt"
text_captions = load_text_captions(filename)
print(text_captions[:357])

1000092795.jpg#0	Two young guys with shaggy hair look at their hands while hanging out in the yard .
1000092795.jpg#1	Two young , White males are outside near many bushes .
1000092795.jpg#2	Two men in green shirts are standing in a yard .
1000092795.jpg#3	A man in a blue shirt standing in a garden .
1000092795.jpg#4	Two friends enjoy time spent together .


In [28]:
# make key:value pairs for filename:5x captions
def load_filename_dict(text_captions):
    dictonary = dict()
    for line in text_captions.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        #first token = image id
        image_id, image_desc = tokens[0], tokens[1:]
        # remove .jpg
        image_id = image_id.split('.')[0]
        # description to string
        image_desc = ' '.join(image_desc)
        if image_id not in dictonary:
            dictonary[image_id] = list()
        dictonary[image_id].append(image_desc)
    return dictonary

descriptions = load_filename_dict(text_captions)
print('Loaded: %d ' % len(descriptions))

Loaded: 31783 


In [29]:
list(descriptions.keys())[:5]

['1000092795', '10002456', '1000268201', '1000344755', '1000366164']

In [30]:
descriptions['1000092795']

['Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 'Two young , White males are outside near many bushes .',
 'Two men in green shirts are standing in a yard .',
 'A man in a blue shirt standing in a garden .',
 'Two friends enjoy time spent together .']

In [33]:
def clean_descriptions(descriptions):
    # translation table for removing punctuation; third param for removal
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            # using trans table to remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove s/a
            desc = [word for word in desc if len(word)>1]
            # words with nos
            desc = [word for word in desc if word.isalpha()]
            desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [34]:
descriptions['1000092795']

['two young guys with shaggy hair look at their hands while hanging out in the yard',
 'two young white males are outside near many bushes',
 'two men in green shirts are standing in yard',
 'man in blue shirt standing in garden',
 'two friends enjoy time spent together']

In [35]:
# vocab words
def to_vocab(descriptions):
    #all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc

# summarize vocabulary
vocabulary = to_vocab(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

Vocabulary Size: 19735


In [36]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

save_descriptions(descriptions, 'dataset/descriptions.txt')

In [3]:
# load train ids
def load_set(filename):
    doc = load_text_captions(filename)
    dataset = list()
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

filename = 'dataset/flickr30k_train.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 29000


In [4]:
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_text_captions(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            # store
            descriptions[image_id].append(desc)
    return descriptions

train_descriptions = load_clean_descriptions('dataset/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=29000


In [5]:
train_descriptions['1000092795']

['startseq two young guys with shaggy hair look at their hands while hanging out in the yard endseq',
 'startseq two young white males are outside near many bushes endseq',
 'startseq two men in green shirts are standing in yard endseq',
 'startseq man in blue shirt standing in garden endseq',
 'startseq two friends enjoy time spent together endseq']

In [6]:
# all training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

145000

In [7]:
# words appearing at least 10 times
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 19062 -> 5227


In [8]:
# inception v3 model
model = InceptionV3(weights='imagenet')

# remove the last layer (output layer) from inception v3; [-1] = last layer's output, [-2] = last layer
model_new = Model(model.input, model.layers[-2].output)

In [9]:
def preprocess(image_path):
    # 299x299 needed by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [11]:
# encode image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [12]:
# all the images
images = 'dataset/flickr30k-images/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [13]:
train_images_file = 'dataset/flickr30k_train.txt'
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [14]:
test_images_file = 'dataset/flickr30k_test.txt'
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [15]:
start = time()
encoding_train = {}
for img in train_img:
    encoding_train[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

Time taken in seconds = 1462.748862504959


In [20]:
# Save bottleneck train features
with open("dataset/Pickle/encoded_train_images.pkl", "wb") as encoded_pickle:
    dump(encoding_train, encoded_pickle)

In [21]:
# for test imgs 
start = time()
encoding_test = {}
for img in test_img:
    encoding_test[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

Time taken in seconds = 62.08194947242737


In [22]:
# Save bottleneck test features
with open("dataset/Pickle/encoded_test_images.pkl", "wb") as encoded_pickle:
    dump(encoding_test, encoded_pickle)

In [23]:
train_features = load(open("dataset/Pickle/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=29000
